In [1]:
!pip install tensorflow

     -------------------------------------- 300.9/300.9 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 442.0/442.0 kB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     ------------------------------------- 938.6/938.6 kB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 8.8 MB/s eta 0:00:00
     -------------------------------------- 413.4/413.4 kB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 8.6 MB/s eta 0:00:00
     -------------------------------------- 133.7/133.7 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 186.9/186

In [2]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [15]:
classes = ["Bengin cases","Malignant cases", "Normal cases"]

In [35]:
from keras.models import load_model
model = load_model("lung_model.h5")
print("Model Loaded!!!")

Model Loaded!!!


In [36]:
import matplotlib.pyplot as plt
def predict_image(img):
    plt.figure(figsize=(40,8))
    print()
    print('-----------------------------------Chest Cancer Type Detection---------------------------------------------------')
    print()
    print('----------------------------------------------RESULT-------------------------------------------------------------')
    print()
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize the image data
    # Predict the class probabilities
    predictions = model.predict(img_array)
    # Get the predicted class label
    predicted_class = np.argmax(predictions)

    # Print the predicted class label
    print("Predicted class:", classes[predicted_class])

In [37]:
from tensorflow.keras.preprocessing import image
import numpy as np
im=image.load_img('the-iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/Malignant cases/Malignant case (101).jpg',target_size=(224,224))
predict_image(im)


-----------------------------------Chest Cancer Type Detection---------------------------------------------------

----------------------------------------------RESULT-------------------------------------------------------------

1/1 [==============================] - 1s 1s/step
Predicted class: Malignant cases


<Figure size 4000x800 with 0 Axes>

In [38]:
im1=image.load_img('the-iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/Normal cases/Normal case (100).jpg',target_size=(224,224))
predict_image(im)


-----------------------------------Chest Cancer Type Detection---------------------------------------------------

----------------------------------------------RESULT-------------------------------------------------------------

1/1 [==============================] - 0s 78ms/step
Predicted class: Malignant cases


<Figure size 4000x800 with 0 Axes>

In [39]:
im1=image.load_img('the-iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/Normal cases/Normal case (1).jpg',target_size=(224,224))
predict_image(im)


-----------------------------------Chest Cancer Type Detection---------------------------------------------------

----------------------------------------------RESULT-------------------------------------------------------------

1/1 [==============================] - 0s 78ms/step
Predicted class: Malignant cases


<Figure size 4000x800 with 0 Axes>

In [27]:
im1=image.load_img('the-iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/Malignant cases/Malignant case (1).jpg',target_size=(256,256))
predict_image(im)

1/1 [==============================] - 0s 167ms/step
Predicted class: 1


In [40]:
im1=image.load_img('the-iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/Bengin cases/Bengin case (1).jpg',target_size=(256,256))
predict_image(im)


-----------------------------------Chest Cancer Type Detection---------------------------------------------------

----------------------------------------------RESULT-------------------------------------------------------------

1/1 [==============================] - 0s 77ms/step
Predicted class: Malignant cases


<Figure size 4000x800 with 0 Axes>

In [ ]:
x=image.img_to_array(img)
    x=x/255
    plt.imshow(img)
    x=np.expand_dims(x,axis=0)

In [4]:
!pip install watchdog

In [6]:
from flask import Flask, render_template, request, redirect, url_for
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import os
import shutil

app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
model = load_model('VGG16_Model.h5')


def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/')
def upload_form():
    return render_template('upload.html')


@app.route('/', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)
    if file and allowed_file(file.filename):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)
        
        # Preprocess the image
        img = image.load_img(file_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = img_array / 255.0
        img_array = img_array.reshape((1, 224, 224, 3))

        # Make prediction
        prediction = model.predict(img_array)
        predicted_class = prediction.argmax(axis=-1)
        
        # Move the uploaded image to the "uploads" folder
        new_file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        shutil.move(file_path, new_file_path)
        
        return redirect(url_for('result', filename=file.filename, predicted_class=predicted_class))
    else:
        return redirect(request.url)


@app.route('/result/<filename>/<predicted_class>')
def result(filename, predicted_class):
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    return render_template('result.html', filename=filename, predicted_class=predicted_class)


if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1